# L1 Language Models, the Chat Format and Tokens

## Tow types of LLMs

### Base LLM

predicts next word by extending input sentence one word at a time iteratively.

Eg, <span style="color: red;">Once upon a time, there was a unicorn</span><br>
that lived in a magical forest with all her unicorn friends. 

Base LLM may output similar questions instead of answering the question when giving a quesiton input. <br>
Eg, <span style="color: red;">What is the capital of France?</span><br>
What is France's largest city? <br>
What is France's population? <br>
What is the currency of France?

### Instruction Tuned LLM
tries to follow instructions. 

Eg, <span style="color: red;">What is the capital of France?</span><br>
The capital of France is Paris. 

### Getting from a Base LLM to an instruction tuned LLM

Train a Base LLM on a lot of data. 

Furhter train the model: 
 - Fine-tune on examples of where the output follows an input instruction 
 - Obtain human-ratings of the quality of different LLM outputs on criteria such as whether it is helpful honest, and harmless. 
 - Tune LLM to increase probability that it generates the more highly rated outputs (using RLHF: Reinforcement Learning from Human Feedback).

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [17]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

#### helper function
This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

## Prompt the model and get a completion

In [3]:
response = get_completion("What is the capital of France?")

In [4]:
print(response)

The capital of France is Paris.


## Tokens

In [5]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

The reversed letters of "lollipop" are "pillipol".


"lollipop" in reverse should be "popillol". 

**This is because LLM predicts the next token, not the next letter.**

Trick: add - to separate each letter.

In [8]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [7]:
response

'p-o-p-i-l-l-o-l'

### Meaning of system, assistant, ans user keywords in prompts
![system-assistant-user](figs/system-assistant-user.png)

## Helper function (chat format)
Here's the helper function we'll use in this course.

In [9]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

In [10]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, the happy carrot in the ground,
With a bright orange color all around.
It grows up tall and straight and true,
Bringing joy to me and you.

Its leaves are green, its roots so strong,
Oh, the happy carrot, singing a song.
With each bite, a crunchy delight,
Filling our bellies with pure delight.

So let's celebrate this joyful root,
The humble carrot, oh how it's cute!
With a smile upon its cheery face,
It adds sweetness to every place.

From gardens to plates, it brings such glee,
The happy carrot, a friend to you and me.
So let's cheer for this vegetable star,
Oh, the happy carrot, we love you, by far!


In [11]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, there was a happy carrot named Charlie who grew up in a vibrant vegetable garden.


In [12]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Once there was a carrot named Larry who was always quite merry, with a smile so bright and a vibrant orange hue, he brought joy to all, yes, even to you!


In [13]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [18]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [19]:
print(response)

Oh, the happy carrot, so bright and orange,
Grown in the garden, a joyful forage.
With a smile so wide, from top to bottom,
It brings happiness, oh how it blossoms!

In the soil it grew, with love and care,
Nurtured by sunshine, fresh air to share.
Its leaves so green, waving in the breeze,
A happy carrot, it aims to please.

With a crunch and a munch, it brings delight,
A healthy snack, oh what a sight!
Filled with vitamins, so good for you,
The happy carrot, it knows what to do.

So let's celebrate this veggie so grand,
With a joyful dance, let's all take a stand.
For the happy carrot, a true delight,
Bringing smiles and happiness, day and night!


In [20]:
print(token_dict)

{'prompt_tokens': 37, 'completion_tokens': 164, 'total_tokens': 201}


#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```bash
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value (less secure, not recommended):

```python
import openai
openai.api_key = "sk-..."
```

More secure: 

```python
from dotenv import load_dotenv, find_dotenv 
_ = load_dotenv(find_dotenv()) # read local .env file 
import os 
import openai 
poenai.api_key = os.getenv('OPENAI_API_KEY')
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.